In [1]:
import numpy as np
import math, random
from  scipy.spatial.distance import cdist
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from Spatial_distributions import *
from Independent_functions import *
from Staples_related import *
#%matplotlib inline

In [2]:
def build_fcc_NP():
    const = 2**1.5*(metal_radius[metal_opt]*scale)
    cells_per_side = int((((2*radius_opt)//const)+1)//2*2+1)
    fcc_block = make_fcc_lattice(const, cells_per_side, cells_per_side, cells_per_side)
    
    fcc_sphere=fcc_block[np.linalg.norm(fcc_block, axis=1)<= (radius_opt-(scale*metal_radius[metal_opt]))]
    bnds = get_bonds_NP(fcc_sphere)
    atid, labs, typs = np.linspace(1, len(fcc_sphere), len(fcc_sphere), dtype = 'int'), ["A"]*len(fcc_sphere), [1]*len(fcc_sphere)
    
    NP = Molecule(atid, fcc_sphere, bnds, labs, typs)
    return NP

def get_bonds_NP(pts):
    bonds = []
    cutoff = 2*scale*metal_radius[metal_opt] + 0.05
    for i in range(len(pts)):
        dist = cdist(pts, pts)[i,:]
        x = np.where(dist <= cutoff)[0]
        #print(dist[dist <= cutoff])
        x = np.delete(x, np.where(x == i)[0])
        x = x + 1
        bonds.append(list(x))
    return bonds

In [3]:
def build_citrate():
    #AB and BAB were taken from atomistic simulations of citrate in water
    AB = 0.288 #bond distance
    BAB = 111.15 #degrees
    A = np.zeros((1,3))
    B1 = [[AB, 0, 0]]
    B2 = AB*np.array([[-np.cos(BAB), np.sin(BAB), 0]])
    
    xyz = np.vstack((A, B1, B2))
    bnds = [[2, 3], [1, 3], [1, 2]]
    atid, labs, typs = np.linspace(1, len(xyz), len(xyz), dtype = 'int'), ["B", "C", "C"], [2, 3, 3]
    
    cit = Molecule(atid, xyz, bnds, labs, typs)
    print("CT-CT bond lenght is: {:.3f}".format(math.sqrt(2*AB**2*(1-math.cos(BAB*math.pi/180)))))
    return cit

In [4]:
def build_sodium(N):
    B = np.array([BX, BY, BZ])
    xyz = np.multiply(np.random.rand(N,3), B) - B/2
    sod = []
    for i in range(N):
        sod.append(Molecule([1], xyz[i], [], ["D"], [4]))
    return sod

In [5]:
def solvate(solute):
    const = (4/8.26)**(1/3) #4 is the num of molecules in FCC lattice. 8.26 is the number density of martini water.
    W = make_fcc_lattice(const, round(BX/const), round(BY/const), round(BZ/const)) #This is usually overestimated
    d = len(W)/(BX*BY*BZ)
    if d > 8.26: 
        random.seed(666)
        while d > 8.26: #additional tolerance
            ndx = random.randint(0, len(W))
            W = np.delete(W, ndx, 0)
            d = len(W)/(BX*BY*BZ)
    else:
        print("Water is not dense enough")
    
    dist = cdist(W, solute.xyz)
    W = W[np.all(dist>=0.5, axis=1)] # 0.5 is somewhere between 2*rH2O and 2*tPt
    
    waters = []
    for i in range(len(W)):
        waters.append(Molecule([1], W[i], [], ["E"], [5]))
    return waters

In [6]:
def init_fort5():
    f5 = open("fort.5", "w")
    f5.write(" box:\n")
    f5.write("  {:>9.6f}\t{:>9.6f}\t{:>9.6f}\t{:>13.6E}\n".format(BX, BY, BZ, BW))
    f5.write(" Numero totale di molecole:\n")
    f5.write("{:>10}\n".format(0))
    f5.close()
    
def add_mol_fort5(mol, molid):
    f5 = open("fort.5", "r")
    f5l = f5.readlines()
    f5l = [l.replace("\n", "") for l in f5l]
    if molid == 1:
        lastid = 0
    else:
        lastid = int(f5l[-1][:4])
    f5.close()
    
    at_per_mol = len(mol.xyz)
    f5 = open("fort.5", "a")
    f5.write("molecola nr.{:>13}\n".format(molid))
    f5.write("{:>7}\n".format(at_per_mol))
    for j in range(at_per_mol):
        atid = lastid+mol.atids[j]
        lab = mol.labels[j]
        tipo = mol.types[j]
        N_bonds = len(mol.bonds[j])
        x, y, z = mol.xyz[j,0] + BX/2, mol.xyz[j,1] + BY/2, mol.xyz[j,2] + BZ/2
        f5.write("{:>4}{:>2}{:>9}{:>7}{:>9.3f}{:>9.3f}{:>9.3f}    ".format(atid, lab, tipo, N_bonds, x, y, z))
        for k in range(12):
            if k >= len(mol.bonds[j]):
                f5.write("{:>7}".format(0))
            else:
                f5.write("{:>7}".format(mol.bonds[j][k]+lastid))
        f5.write("\n")
    f5.close()
    
def add_wat_fort5(wats, prev_molid):
    f5 = open("fort.5", "r")
    f5l = f5.readlines()
    f5l = [l.replace("\n", "") for l in f5l]
    lastid = int(f5l[-1][:4])
    f5.close()
    
    molid = prev_molid + 1
    N_mol = len(wats)
    f5 = open("fort.5", "a")
    for i in range(N_mol):
        f5.write("molecola nr.{:>13}\n".format(molid))
        f5.write("{:>7}\n".format(1))
        atid = lastid+wats[i].atids[0]
        lab = wats[i].labels[0]
        tipo = wats[i].types[0]
        N_bonds = len(wats[i].bonds)
        x, y, z = wats[i].xyz[0] + BX/2, wats[i].xyz[1] + BY/2, wats[i].xyz[2] + BZ/2
        f5.write("{:>4}{:>2}{:>9}{:>7}{:>9.3f}{:>9.3f}{:>9.3f}    ".format(atid, lab, tipo, N_bonds, x, y, z))
        for k in range(12):
            f5.write("{:>7}".format(0))
        f5.write("\n")
        molid = molid + 1
        lastid = lastid +1
    f5.close()

def terminate_fort5():
    new_lines = []
    f5 = open("fort.5", "r")
    f5l = f5.readlines()
    f5.close()

    f5l_copy = f5l
    for l, line in enumerate(f5l):
        if "molecola nr." in line:
            N_mol = int(f5l[l][-6:])
    for l, line in enumerate(f5l_copy):
        if "Numero totale di molecole" in line:
            f5l_copy[l+1] = "{:>6}\n".format(N_mol)
            break
            
    f5 = open("fort.5", "w")
    for line in f5l_copy:
        f5.write(line)
            

In [7]:
def write_fort3():
    f3 = open("fort.3", "w")
    f3.write("********* model file ******************************\n")
    
    f3.write("{:>2}   different atom types\n".format(len(atypes)))
    f3.write("*label    mass     charge\n")
    for a in atypes:
        f3.write("{:>2}{:>6}{:>12.4f}{:>12.4f}\n".format(a[0], a[1], a[2], a[3]))
    f3.write("****************************************************\n")
    
    f3.write("{:>2}   different bond types\n".format(len(btypes)))
    f3.write("*atom 1   atom2   bond_length   force_constant\n")
    for b in btypes:
        f3.write("{:>2}{:>10}{:>12.4f}{:>12.4f}\n".format(b[0], b[1], b[2], b[3]))
    f3.write("****************************************************\n")

    f3.write("{:>2}   different bond angles\n".format(len(antypes)))
    f3.write("*atom1   atom2    atom3     theta0(deg)  force_constant\n")
    for c in antypes:
        f3.write("{:>2}{:>10}{:>10}{:>12.4f}{:>12.4f}\n".format(c[0], c[1], c[2], c[3], c[4]))
    f3.write("****************************************************\n")

    f3.write("{:>2}   different torsions\n".format(len(ttypes)))
    f3.write("*atom1***atom2*** atom3***atom4***\n")
    for d in ttypes:
        f3.write("{:>2}{:>10}{:>10}{:>10}\n".format(d[0], d[1], d[2], d[3]))
    f3.write("****************************************************\n")

    f3.write("{:>2}   different non-bonded interactions\n".format(len(vdw)))
    f3.write("*type 1 ** type 2 sigma  epsilon\n")
    for v in vdw:
        f3.write("{:>2}{:>10}{:>12.4f}{:>12.4f}\n".format(v[0], v[1], v[2], v[3]))
        
    f3.write("***** SCF settings **********************************\n")
    f3.write("* mx   my   mz  cells in  X Y Z directions\n")
    f3.write("{:>7}{:>7}{:>7}\n".format(cells[0], cells[1], cells[2]))
    
    f3.write("* compressibility\n")
    f3.write("{:>5.2f}\n".format(compress))

    f3.write("*chi matrix corresponding test Q for PEO and test H for PPO\n")
    for row in chi:
        for x in row:
            f3.write("{:>7.2f}".format(x))
        f3.write("\n")

# Running code

## Structure

### General

In [8]:
BX, BY, BZ, BW = 7.0, 7.0, 7.0, 0.0 #nm
SYS = System(BX, BY, BZ)
init_fort5()

### Nanoparticles

In [9]:
metal_radius = {'Pt' : 0.1385} #in nm
scale = 1.5   #Coarse-graining level for the metal
metal_opt = "Pt"
lignum_opt = 0
radius_opt = 1.25 #nm

In [10]:
NP = build_fcc_NP()
SYS.add_molecule(NP)
add_mol_fort5(NP, 1)
print("Scaling:", scale, "Beads:", len(NP.xyz))

Scaling: 1.5 Beads: 92


### Citrate

In [11]:
N_cit = 10
all_xyz = NP.xyz
for i in range(N_cit):
    CIT = build_citrate()
    CIT.remove_clashes(SYS, BX, BY, BZ)
    SYS.add_molecule(CIT)
    add_mol_fort5(CIT, i + 2)

CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475
CT-CT bond lenght is: 0.475


### Sodium

In [12]:
N_sod = 3*N_cit
NA = build_sodium(N_sod)
for i in range(len(NA)):
    SYS.add_molecule(NA[i])
add_wat_fort5(NA, 1 + N_cit)

(None,)

### Water

In [13]:
W = solvate(NP)
for i in range(len(W)):
    SYS.add_molecule(W[i])
add_wat_fort5(W, 1 + N_cit + N_sod)

In [14]:
terminate_fort5()

In [21]:
print_xyz("1NP5CITNAWAT", SYS.xyz, SYS.labels)

## Parameters

In [19]:
res      = 0.65 #nm/cell
compress = 0.05

atypes   = [[1,  "A",   784.238,   0.0],\
            [2,  "B",      72.0,  -1.0],\
            [3,  "C",      58.0,  -1.0],\
            [4,  "D",      72.0,   0.0],\
            [5,  "E",      72.0,   0.0]\
           ]
btypes   = [[1,    1, 0.4155,   5000],\
            [2,    3,  0.288,   5000],\
            [3,    3,  0.475,   5000],\
           ]
antypes  = [\
           ]
ttypes   = [\
           ]
vdw      = [[1,    1,    666,   666],\
            [2,    2,    666,   666],\
            [3,    3,    666,   666],\
            [4,    4,    666,   666],\
            [5,    5,    666,   666],\
           ]
#Order is     Pt,   CM,   CT,   NA,   W
chi      = [[ 0.0,  0.0,  0.0,  0.0,  0.0],\
            [ 0.0,  0.0,  0.0,  0.0,  0.0],\
            [ 0.0,  0.0,  0.0,  0.0, 10.0],\
            [ 0.0,  0.0,  0.0,  0.0,  0.0],\
            [ 0.0,  0.0, 10.0,  0.0,  0.0],\
           ]

cells    = list(map(int, [BX//res, BY//res, BZ//res]))

#The mass for particles A is 195.084 * N(CG)/N(AA)

In [20]:
write_fort3()